In [2]:
import pandas as pd
import numpy as np
import os.path
import recsys as rs
import notipy
import math
import sys
import TopSimilarRecommender as TSR
import ItemBasedRecommender as IBR
import UserBasedRecommender as UBR
import random
from scipy import sparse as sps
import importlib

In [3]:
train = pd.read_csv('Data/train_final.csv','\t')
tracks = pd.read_csv('Data/tracks_final.csv','\t')
pl_info = pd.read_csv('Data/playlists_final.csv','\t')
tgt_playlists = pd.read_csv('Data/target_playlists.csv','\t')
tgt_tracks = pd.read_csv('Data/target_tracks.csv','\t')

In [3]:
#train, test, tgt_tracks, tgt_playlists = rs.split_train_test(train, 10, 20, 5, 2517)
#tgt_playlists.shape

In [4]:
ibr = IBR.ItemBasedRecommender(idf=True, shrinkage=10,n_el_sim=65)
ibr.fit(tracks,train,tgt_tracks,save_sim=True)#saved_similarity='BuiltStructures/ibr_sim_65el_h10_idfTrue.npz'
print('Model fitted!')

  0%|          | 20/45649 [00:00<03:53, 195.22it/s]

Calculated Indices


100%|██████████| 45649/45649 [04:00<00:00, 189.83it/s]


(45649, 100000)
Model URM built
Model URM regularized with IDF!


100%|██████████| 100000/100000 [45:12<00:00, 36.87it/s]


(100000, 32195)
Model fitted!


In [5]:
tsr= TSR.TopSimilarRecommender(attributes=['artist_id', 'album', 'playcount'],idf=True, n_min_attr=90, n_el_sim=65)
tsr.fit(tracks,tgt_tracks,save_sim=True)# saved_similarity='BuiltStructures/tsr_sim_65el_h10_idfTrue.npz'
print('Model fitted!')

Fixed dataset
Calculated Indices


  0%|          | 32/100000 [00:00<05:24, 307.87it/s]

ICM built
ICM regularized with IDF!


100%|██████████| 100000/100000 [28:09<00:00, 59.18it/s]


Similarity built
Model fitted!


In [29]:
ubr = UBR.UserBasedRecommender(measure='imp_cos', shrinkage=20,n_el_sim =65)
ubr.fit(tracks,train,tgt_playlists, save_sim=True)
print('Model fitted!')


  0%|          | 17/45649 [00:00<04:36, 164.95it/s]

Calculated Indices



  1%|          | 501/45649 [00:02<03:44, 200.99it/s]
Exception in thread Thread-21:
Traceback (most recent call last):
  File "/home/giada/anaconda3/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/home/giada/anaconda3/lib/python3.5/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/giada/anaconda3/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

  0%|          | 8/45649 [00:00<10:21, 73.48it/s]

Model URM built


100%|██████████| 45649/45649 [09:29<00:00, 80.15it/s]


Similarity built
Model fitted!


# Recommendation

In [31]:
_, _, IX_tgt_playlists, _ = rs.create_sparse_indexes(playlists=tgt_playlists)
_, _, IX_playlists, _ = rs.create_sparse_indexes(tracks_info=tracks, playlists=train, tracks_reduced=tracks)
URM = rs.create_tgt_URM(IX_tgt_playlists, tsr.IX_items, train)
URM_UBR =rs.create_UBR_URM(IX_playlists, tsr.IX_tgt_items, train)
print('URM built')

100%|██████████| 45649/45649 [09:55<00:00, 76.61it/s]


URM built


In [34]:
ubr.S =ubr.S.T.tocsr()
URM = URM.tocsr()
tsr.S = tsr.S.tocsr()
ibr.S =ibr.S.tocsr()
URM_UBR= URM_UBR.tocsr()

In [35]:
rs = importlib.reload(rs)
recommendetions = np.array([])
div_t = tsr.S.sum(axis=0)
div_i = ibr.S.sum(axis=0)
H_t=15
H_i=30
for p in IX_tgt_playlists.values:
    avg_sims_t = URM[p,:].dot(tsr.S).toarray().ravel()
    avg_sims_t = avg_sims_t/(div_t+H_t+1e-6)
    avg_sims_i = URM[p,:].dot(ibr.S).toarray().ravel()
    avg_sims_i = avg_sims_i/(div_i+H_i+1e-6)
    avg_sims_u = ubr.S[p,:].dot(URM_UBR).toarray().ravel()
    avg_sims = np.array(avg_sims_t*0.6 + avg_sims_i*0.2+avg_sims_u*0.2).ravel()
    top = rs.top5_outside_playlist(avg_sims, p, train, IX_tgt_playlists, tsr.IX_tgt_items, False, False)
    recommendetions = np.append(recommendetions, rs.sub_format(top))
    if (p % 1000 == 0):
        print('Recommended ' + str(p) + ' users over ' + str(IX_tgt_playlists.values.shape[0]))
rec_tsr =  pd.DataFrame({'playlist_id' : IX_tgt_playlists.index.values, 'track_ids' : recommendetions})

(10000, 100000)
Recommended 0 users over 10000
Recommended 1000 users over 10000
Recommended 2000 users over 10000
Recommended 3000 users over 10000
Recommended 4000 users over 10000
Recommended 5000 users over 10000
Recommended 6000 users over 10000
Recommended 7000 users over 10000
Recommended 8000 users over 10000
Recommended 9000 users over 10000


# Evaluation

In [129]:
def evaluate(results, test, eval_metric='MAP'):
    if eval_metric == 'MAP':
        APs = results.apply(calculate_AP, axis=1, args=(test,))
        res = (APs.sum())/results.shape[0]
    return res

def calculate_AP(row, test):
    p_id = row['playlist_id']
    recs = np.fromstring(row['track_ids'], dtype=float, sep=' ')

    AP = 0
    rel_sum = 0
    n_rel_items = min(test[test['playlist_id'] == p_id].shape[0],5)
    for i in range(recs.size):
        rel = 1 if ((test['playlist_id'] == int(p_id)) & (test['track_id'] == recs[i])).any() else 0
        rel_sum += rel
        P = rel_sum/(i+1)
        AP += (P * rel)/n_rel_items

    return AP

In [130]:
map_eval = evaluate(rec_tsr, test, 'MAP')
print('Evaluation completed!')

Evaluation completed!


In [131]:
map_eval

0.10056033182503786

# Save to csv

In [36]:
rec_tsr.to_csv('Submissions/ensemble_submission_notags_alpha060202_h1530.csv', index=False)
print('Results saved as csv!')

Results saved as csv!
